In [6]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
from query_to_mql_tt_integrated import system_msg,context

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [14]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [15]:
model_name = "NousResearch/llama-2-7b-chat-hf"
BASE_MODEL = model_name
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
    device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 15.60 GiB total capacity; 14.74 GiB already allocated; 44.94 MiB free; 14.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [21]:
#Testing prompt

# question = "what are the sales in africa"
# prompt = f"""[INST]<<SYS>>
# Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.
# For example if user gives "top  orders in march 2020", you should only return march 2020.

# <</SYS>>
# {question}
# [/INST]
# [END]"""

In [7]:
def gereneate_output_llama2(prompt):
    prompt = json.dumps(prompt)
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    #model.eval()
    response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
    return response

In [8]:
def get_user_question(question):
    return "Based on the following context:\n" + str(context) + "\n return a response for the question:\n" + question + "\n"

In [9]:
def compose_prompt(question):
    return f"""[INST]<<SYS>>
            {system_msg}
            <</SYS>>
            {get_user_question(question)}
            [/INST]"""

In [10]:
question = input("Enter any question:" )
gereneate_output_llama2(compose_prompt(question))

Enter any question:discount of categories in corporate


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 524.00 MiB (GPU 0; 15.60 GiB total capacity; 14.35 GiB already allocated; 88.94 MiB free; 14.84 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF